# 물질의 임계온도 예측 모델 개발
주최 : 과학기술정보통신부, 정보통신산업진흥원

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as lin
import random
import math
import tensorflow as tf
import seaborn as sns
import scipy as sp
import scipy.stats
import keras
import sklearn
import missingno as msno
import warnings

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from keras.callbacks import ModelCheckpoint, EarlyStopping

warnings.filterwarnings('ignore') # 워닝 메세지를 생략해 줍니다.
pd.set_option('display.max_row', 100000)
pd.set_option('display.max_columns', 100000)

In [ ]:
df = pd.read_csv('./train.csv')
print(df)
print(df.shape)

df.info()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
plt.style.use('seaborn')
sns.set(font_scale=2.5)
%matplotlib inline

In [ ]:
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')
output_sample = pd.read_csv('./output_sample.csv')

# drop = [80]

# print(df_train.head(5))

# for i in drop:
#  df_train = df_train.drop("feature_%d"%(i), axis = 1)
#  print(i)
#  df_test = df_test.drop("feature_%d"%(i), axis = 1)


# 1.1 결측치(Null Data) 확인
# df_train.isnull().sum() / df_train.shape[0]

# df_test.isnull().sum() / df_test.shape[0]

# df_train.head(5)
# df_test.head(5)

# 데이터 세트 분리하기(train, test)

df_train['feature_0'] = df_train['feature_0'].astype('float64')
df_train['feature_27'] = df_train['feature_27'].astype('float64')
df_train['feature_77'] = df_train['feature_77'].astype('float64')

df_test['feature_0'] = df_test['feature_0'].astype('float64')
df_test['feature_27'] = df_test['feature_27'].astype('float64')
df_test['feature_77'] = df_test['feature_77'].astype('float64')

X = df_train.iloc[:, 0:-1]
y = df_train.iloc[:, [-1]]

print(X.shape)
print(y.shape)
print(df_test.shape)

(14884, 81)
(14884, 1)
(6379, 81)


# 데이터의 안에 있는 int를 float로 수정

In [ ]:
# 데이터 형변환

# int를 float타입으로 바꿈
# df_train['feature_0'] = df_train['feature_0'].apply(lambda x: x.replace(',', '')).astype('float')
df_train['feature_0'] = df_train['feature_0'].astype('float64')
df_train['feature_27'] = df_train['feature_27'].astype('float64')
df_train['feature_77'] = df_train['feature_77'].astype('float64')

df_test['feature_0'] = df_test['feature_0'].astype('float64')
df_test['feature_27'] = df_test['feature_27'].astype('float64')
df_test['feature_77'] = df_test['feature_77'].astype('float64')

# 데이터 전처리가 끝났으므로 csv 파일로 저장
df_train.to_csv("/content/train.csv", mode='w', index=False)
df_test.to_csv("/content/test.csv", mode='w', index=False)

## PCA로 분류하기

In [ ]:
 print(X.head(20))

    feature_0   feature_1   feature_2   feature_3   feature_4  feature_5  \
0         5.0   72.324650   49.654329   58.637102   34.476736   1.444612   
1         3.0   49.324473   76.452769   41.291894   68.270993   0.917960   
2         3.0   18.038134   15.418165   17.090687   14.269447   1.048744   
3         5.0   61.176081   71.795475   44.686874   55.579038   1.339611   
4         4.0   91.319175   82.062345   59.066285   49.844388   1.069152   
5         5.0   62.001281   72.886955   44.947007   56.176451   1.332068   
6         2.0  119.086881  119.086881   80.110945   80.110945   0.386476   
7         2.0  203.895000  205.067742  203.868212  205.044282   0.693016   
8         2.0   72.933500   91.733375   68.490627   89.603989   0.632864   
9         5.0   89.337180   53.602733   70.560647   36.207043   1.444537   
10        4.0   77.437517   63.148706   69.471101   60.905485   1.276404   
11        2.0   60.332250   55.373934   59.596931   54.857947   0.680984   
12        5.

In [ ]:
#모델 생성 및 적용
pca = PCA()
pca.fit(X)
pca.transform(X)

array([[-4.56233278e+03,  1.73807273e+02,  1.97272426e+02, ...,
        -2.08197203e-03, -9.57384454e-04, -3.55943989e-03],
       [ 3.34294376e+03, -2.45921039e+03,  4.01423337e+03, ...,
        -5.77466957e-03, -6.49830720e-03,  2.34549198e-03],
       [-5.19065303e+03, -8.10575075e+03,  1.57388148e+03, ...,
         2.03956329e-02,  2.50350062e-02,  1.06943974e-02],
       ...,
       [-4.73210589e+03,  1.16528281e+03, -9.70626723e+02, ...,
         1.04209678e-03, -3.62809623e-03, -9.53179196e-05],
       [-2.17387537e+03,  1.36081765e+03,  7.24300898e+02, ...,
        -1.67141711e-03, -1.96924031e-03,  2.41503841e-03],
       [ 6.34187764e+03, -4.72419593e+03, -2.91721680e+03, ...,
        -3.03784637e-04,  3.42696119e-03, -3.50451744e-03]])

In [ ]:
print(X.head(20))

    feature_0   feature_1   feature_2   feature_3   feature_4  feature_5  \
0         5.0   72.324650   49.654329   58.637102   34.476736   1.444612   
1         3.0   49.324473   76.452769   41.291894   68.270993   0.917960   
2         3.0   18.038134   15.418165   17.090687   14.269447   1.048744   
3         5.0   61.176081   71.795475   44.686874   55.579038   1.339611   
4         4.0   91.319175   82.062345   59.066285   49.844388   1.069152   
5         5.0   62.001281   72.886955   44.947007   56.176451   1.332068   
6         2.0  119.086881  119.086881   80.110945   80.110945   0.386476   
7         2.0  203.895000  205.067742  203.868212  205.044282   0.693016   
8         2.0   72.933500   91.733375   68.490627   89.603989   0.632864   
9         5.0   89.337180   53.602733   70.560647   36.207043   1.444537   
10        4.0   77.437517   63.148706   69.471101   60.905485   1.276404   
11        2.0   60.332250   55.373934   59.596931   54.857947   0.680984   
12        5.

# 데이터 정규화 MinMax정규화에서 0.5만큼 변환했음 ->  중간으로 더 수렴함

In [ ]:
scaler_x=MinMaxScaler()
scaler_x.fit(X)
Normalization_data_x=scaler_x.transform(X)

train_xMM = pd.DataFrame(data=Normalization_data_x)

scaler_y=MinMaxScaler()
scaler_y.fit(y)
train_Normalization_data_y=scaler_y.transform(y)

train_yMM = pd.DataFrame(data=train_Normalization_data_y)

test_Normalization_data=scaler_x.transform(df_test)

real_test_xMM = pd.DataFrame(data=test_Normalization_data)

print(train_Normalization_data_y.shape)
print(y.shape)

(14884, 1)
(14884, 1)


# train 데이터랑 validation_test 데이터랑 나눔

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(train_xMM, train_yMM, test_size = 0.2, random_state=0, shuffle = True)

In [ ]:
print(train_x.shape)
print(train_y.shape)

print(test_x.shape)
print(test_y.shape)

(11907, 81)
(11907, 1)
(2977, 81)
(2977, 1)


# 선형 회귀
128 -> 64 -> 32 -> 1로 줄어듬

최적화 함수 = adam

손실 함수 = Meen Sqared Error

metrics = MeanAbsoluteError

learning_rate = 학습을 어느 크기로 진행할것인가(batch_size의 크기에서 얼마 단위로 학습하기 때문에 batch_size를 줄이면 같이 줄어듬)

In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(256, activation = 'tanh', input_shape = (train_x.shape)),
#     tf.keras.layers.Dense(128, activation = "tanh"),
#     tf.keras.layers.Dense(64, activation = "tanh"),
#     tf.keras.layers.Dense(8, activation = "elu"),
#     tf.keras.layers.Dense(1)
# ])
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1024, activation = 'tanh', input_shape = (train_x.shape)),
    tf.keras.layers.Dense(500, activation = "tanh"),
    tf.keras.layers.Dense(250, activation = "tanh"),
    tf.keras.layers.Dense(180, activation = "tanh"),
    tf.keras.layers.Dense(50, activation = "tanh"),
    tf.keras.layers.Dense(4, activation = "elu"),
    tf.keras.layers.Dense(1)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer = optimizer, loss = 'mae', metrics = [tf.keras.metrics.MeanAbsoluteError()])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 11907, 1024)       83968     
_________________________________________________________________
dense_1 (Dense)              (None, 11907, 500)        512500    
_________________________________________________________________
dense_2 (Dense)              (None, 11907, 250)        125250    
_________________________________________________________________
dense_3 (Dense)              (None, 11907, 180)        45180     
_________________________________________________________________
dense_4 (Dense)              (None, 11907, 50)         9050      
_________________________________________________________________
dense_5 (Dense)              (None, 11907, 4)          204       
_________________________________________________________________
dense_6 (Dense)              (None, 11907, 1)          5

# 위에서 짠 네트워크를 학습하느 부분 Early_stopping을 걸어놓음 
early_stoppping
moniter = 무엇을 참고할지

min_delta = 얼마나 변해야지 변했다고 인지할것인가

patience =  몇번 안변했을때 스탑시킬것인가?

bath_size = 얼마만큼 짤라서 학습을 할것인가?



In [ ]:
early_stopping = EarlyStopping(monitor = 'val_mean_absolute_error', min_delta = 0.00001, patience = 50, verbose =1, mode = 'auto', baseline = None, restore_best_weights = True)
history = model.fit(train_x, train_y, epochs = 10000, validation_split = 0.25, batch_size = 128 ,callbacks=(early_stopping))

Epoch 1/10000
70/70 [==============================] - 3s 32ms/step - loss: 0.2466 - mean_absolute_error: 0.2466 - val_loss: 0.0833 - val_mean_absolute_error: 0.0833
Epoch 2/10000
70/70 [==============================] - 0s 5ms/step - loss: 0.0814 - mean_absolute_error: 0.0814 - val_loss: 0.0677 - val_mean_absolute_error: 0.0677
Epoch 3/10000
70/70 [==============================] - 0s 6ms/step - loss: 0.0714 - mean_absolute_error: 0.0714 - val_loss: 0.0749 - val_mean_absolute_error: 0.0749
Epoch 4/10000
70/70 [==============================] - 0s 6ms/step - loss: 0.0720 - mean_absolute_error: 0.0720 - val_loss: 0.0642 - val_mean_absolute_error: 0.0642
Epoch 5/10000
70/70 [==============================] - 0s 6ms/step - loss: 0.0660 - mean_absolute_error: 0.0660 - val_loss: 0.0711 - val_mean_absolute_error: 0.0711
Epoch 6/10000
70/70 [==============================] - 0s 6ms/step - loss: 0.0642 - mean_absolute_error: 0.0642 - val_loss: 0.0603 - val_mean_absolute_error: 0.0603
Epoch 7/1

#Epoch 1095/10000  batch_size = 1024
9/9 [==============================] - 0s 11ms/step - loss: 0.0409 -
mean_absolute_error: 0.0409 - val_loss: 0.0430 - val_mean_absolute_error: 0.0430

#Epoch 118/10000 batch_size = 2048
5/5 [==============================] - 0s 12ms/step - loss: 0.0396 - mean_absolute_error: 0.0396 - val_loss: 0.0426 - val_mean_absolute_error: 0.0426
Restoring model weights from the end of the best epoch.


# 결과를 출력하는 부분

In [ ]:
test = model.predict(real_test_xMM)

predict_data = scaler_y.inverse_transform(test)
print(predict_data)

outputdf = pd.DataFrame(predict_data, columns=['target'])
print(outputdf)
outputdf.to_csv('/content/output_sample.csv', index=False)
print(1)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1380   68.504486
1381   13.333478
1382    4.778013
1383    1.509501
1384    3.320187
1385   69.715065
1386   57.757622
1387    2.223589
1388    6.241419
1389   80.396873
1390   27.011753
1391    6.469499
1392   68.796913
1393   40.539730
1394   78.388542
1395    3.083529
1396  114.207893
1397    6.321198
1398  119.784920
1399   44.549202
1400   63.901855
1401   68.369438
1402    2.939860
1403   39.527267
1404   29.102179
1405   21.985760
1406    4.047832
1407    4.227266
1408    5.076322
1409   81.735535
1410   61.086525
1411    1.806587
1412    4.009309
1413   11.994891
1414    5.121223
1415    3.714132
1416   91.999222
1417   90.896866
1418  103.975281
1419    3.872367
1420    4.378135
1421    4.436313
1422   15.436888
1423   80.013405
1424   41.653984
1425   79.557076
1426   19.364172
1427   59.821293
1428    1.871822
1429    2.746118
1430    7.948154
1431   10.794729
1432    2.370941
1433    1.537432
1434   10.605525
1435    3.791342
1436    3.44